# Tutorial 2: Building Agents

**Goal:** Create the complete 3-level HERON agent hierarchy (Field, Coordinator, System).

**Time:** ~20 minutes

---

## Agent Hierarchy

HERON uses a 3-level hierarchy where each level has different timing:

```
Level 3: SystemAgent (grid operator)      — tick_interval: 300s
    │
Level 2: CoordinatorAgent (microgrid)     — tick_interval: 60s
    │
Level 1: FieldAgent (battery, generator)  — tick_interval: 1s
```

**Why timing matters:**
- In real systems, devices update fast, coordinators update slower
- HERON captures this with `tick_config` on each agent
- Enables realistic event-driven testing (Tutorial 05)

## Step 1: Building a Field Agent (Battery)

Field agents control physical devices. The **essential HERON pattern** is:

1. **Pass features to constructor** - Features are provided as a list
2. **Implement required methods:**
   - `init_action()` - Define action space
   - `set_action()` - Store action values
   - `set_state()` - Denormalize action and update features
   - `apply_action()` - Call `set_state()` to apply action
   - `compute_local_reward()` - Compute reward from local state

```python
class MyFieldAgent(FieldAgent):
    def __init__(self, agent_id, ...):
        features = [MyFeature(...)]  # Create features
        super().__init__(agent_id=agent_id, features=features, ...)
    
    def init_action(self, features) -> Action:  # Define action space
        ...
    def set_action(self, action):               # Store action
        ...
    def set_state(self, **kwargs):              # Update features
        ...
    def apply_action(self):                     # Call set_state()
        self.set_state()
    def compute_local_reward(self, local_state) -> float:
        ...
```

**Important:** Agents don't have `step()`. Physics/simulation lives in the **environment**.

In [1]:
import numpy as np
from dataclasses import dataclass
from typing import Any, ClassVar, List, Optional, Sequence

from heron.agents.field_agent import FieldAgent
from heron.core.feature import FeatureProvider
from heron.core.action import Action
from heron.scheduling.tick_config import TickConfig


# Features from Tutorial 1
@dataclass(slots=True)
class BatterySOC(FeatureProvider):
    """Battery state of charge - visible to owner and coordinator."""
    visibility: ClassVar[Sequence[str]] = ['owner', 'upper_level']
    soc: float = 0.5

    def set_values(self, **kw):
        if 'soc' in kw:
            self.soc = float(np.clip(kw['soc'], 0.0, 1.0))


@dataclass(slots=True)
class CostSafety(FeatureProvider):
    """Cost and safety metrics for reward computation."""
    visibility: ClassVar[Sequence[str]] = ['owner', 'upper_level']
    cost: float = 0.0
    safety: float = 0.0

In [2]:
class SimpleBattery(FieldAgent):
    """Battery agent - demonstrates HERON FieldAgent pattern.
    
    Action: power setpoint [-1, 1] where negative=discharge, positive=charge
    """
    
    def __init__(
        self,
        agent_id: str,
        capacity_MWh: float = 2.0,
        max_power_MW: float = 0.5,
        init_soc: float = 0.5,
        upstream_id: Optional[str] = None,
        tick_config: Optional[TickConfig] = None,
    ):
        # Store device parameters
        self._capacity = capacity_MWh
        self._max_power = max_power_MW
        self._init_soc = init_soc
        
        # Create features (passed to parent)
        features = [
            BatterySOC(soc=init_soc),
            CostSafety(cost=0.0, safety=0.0),
        ]
        
        super().__init__(
            agent_id=agent_id,
            features=features,
            upstream_id=upstream_id,
            tick_config=tick_config or TickConfig.deterministic(tick_interval=1.0),
        )
    
    # === REQUIRED: init_action ===
    def init_action(self, features: List[FeatureProvider] = []) -> Action:
        """Define normalized [-1, 1] action space."""
        action = Action()
        action.set_specs(
            dim_c=1,  # 1D continuous action
            range=(np.array([-1.0]), np.array([1.0])),
        )
        action.set_values(c=np.zeros(1, dtype=np.float32))
        return action
    
    # === REQUIRED: set_action ===
    def set_action(self, action: Any, *args, **kwargs) -> None:
        """Store action values."""
        if isinstance(action, Action):
            self.action.set_values(c=action.c)
        elif isinstance(action, np.ndarray):
            self.action.set_values(c=action)
        else:
            raise ValueError(f"Unsupported action type: {type(action)}")
    
    # === REQUIRED: set_state ===
    def set_state(self, **kwargs) -> None:
        """Denormalize action and update features.
        
        This is where the agent's physics/dynamics happen.
        """
        # Get current SOC
        soc_feature = self.state.features['BatterySOC']
        cost_feature = self.state.features['CostSafety']
        
        # Denormalize action: [-1, 1] -> [-max_power, max_power]
        p_norm = self.action.c[0] if self.action.c.size > 0 else 0.0
        power_MW = p_norm * self._max_power
        
        # Update SOC (simplified physics)
        dt = 1.0  # hour
        delta_soc = power_MW * dt / self._capacity
        new_soc = np.clip(soc_feature.soc + delta_soc, 0.1, 0.9)
        soc_feature.set_values(soc=new_soc)
        
        # Update cost/safety (battery has no fuel cost, just SOC violations)
        safety = max(0.0, 0.1 - new_soc) + max(0.0, new_soc - 0.9)
        cost_feature.set_values(cost=0.0, safety=safety)
    
    # === REQUIRED: apply_action ===
    def apply_action(self) -> None:
        """Apply the current action to update state."""
        self.set_state()
    
    # === REQUIRED: compute_local_reward ===
    def compute_local_reward(self, local_state: dict) -> float:
        """Compute reward from local state dict.
        
        Args:
            local_state: Dict mapping feature_name -> np.array vector
        
        Returns:
            Reward value (higher is better)
        """
        cost = 0.0
        safety = 0.0
        
        if 'CostSafety' in local_state:
            metrics = local_state['CostSafety']
            cost = float(metrics[0])
            safety = float(metrics[1])
        
        return -cost - safety  # Minimize cost and safety violations

In [3]:
# Test SimpleBattery
battery = SimpleBattery(
    agent_id='bat_1',
    capacity_MWh=2.0,
    max_power_MW=0.5,
    init_soc=0.5,
    upstream_id='mg_1',
)

print(f"Agent: {battery.agent_id}")
print(f"Action space: {battery.action.space}")
print(f"Features: {list(battery.state.features.keys())}")
print(f"Initial SOC: {battery.state.features['BatterySOC'].soc}")

# Test the action flow
battery.set_action(np.array([0.5]))  # Charge at 50% power
battery.apply_action()  # This calls set_state()

print(f"\nAfter action:")
print(f"  SOC: {battery.state.features['BatterySOC'].soc:.3f}")
print(f"  State vector: {battery.state.vector()}")

Agent: bat_1
Action space: Box(-1.0, 1.0, (1,), float32)
Features: ['BatterySOC', 'CostSafety']
Initial SOC: 0.5

After action:
  SOC: 0.625
  State vector: [0.625 0.    0.   ]


## Step 2: Building a Generator Agent

Same pattern, different domain logic.

In [4]:
@dataclass(slots=True)
class GenOutput(FeatureProvider):
    """Generator output - visible to owner, coordinator, and system."""
    visibility: ClassVar[Sequence[str]] = ['owner', 'upper_level', 'system']
    p_mw: float = 0.0
    p_max: float = 5.0

    def vector(self) -> np.ndarray:
        """Return normalized power."""
        return np.array([self.p_mw / max(self.p_max, 1e-6)], dtype=np.float32)


class SimpleGenerator(FieldAgent):
    """Generator agent - same HERON pattern as battery."""
    
    def __init__(
        self,
        agent_id: str,
        p_max_MW: float = 5.0,
        cost_per_MWh: float = 50.0,
        upstream_id: Optional[str] = None,
        tick_config: Optional[TickConfig] = None,
    ):
        self._p_max = p_max_MW
        self._cost_per_MWh = cost_per_MWh
        
        features = [
            GenOutput(p_mw=0.0, p_max=p_max_MW),
            CostSafety(cost=0.0, safety=0.0),
        ]
        
        super().__init__(
            agent_id=agent_id,
            features=features,
            upstream_id=upstream_id,
            tick_config=tick_config or TickConfig.deterministic(tick_interval=1.0),
        )
    
    def init_action(self, features: List[FeatureProvider] = []) -> Action:
        """Action: power output [0, 1] normalized."""
        action = Action()
        action.set_specs(
            dim_c=1,
            range=(np.array([0.0]), np.array([1.0])),  # Generator: 0 to max
        )
        action.set_values(c=np.zeros(1, dtype=np.float32))
        return action
    
    def set_action(self, action: Any, *args, **kwargs) -> None:
        if isinstance(action, Action):
            self.action.set_values(c=action.c)
        elif isinstance(action, np.ndarray):
            self.action.set_values(c=action)
    
    def set_state(self, **kwargs) -> None:
        gen_feature = self.state.features['GenOutput']
        cost_feature = self.state.features['CostSafety']
        
        # Denormalize: [0, 1] -> [0, p_max]
        p_norm = self.action.c[0] if self.action.c.size > 0 else 0.0
        power_MW = p_norm * self._p_max
        
        gen_feature.set_values(p_mw=power_MW)
        
        # Fuel cost
        cost = power_MW * self._cost_per_MWh
        cost_feature.set_values(cost=cost, safety=0.0)
    
    def apply_action(self) -> None:
        self.set_state()
    
    def compute_local_reward(self, local_state: dict) -> float:
        cost = 0.0
        if 'CostSafety' in local_state:
            cost = float(local_state['CostSafety'][0])
        return -cost


# Test
gen = SimpleGenerator('gen_1', p_max_MW=10.0)
gen.set_action(np.array([0.8]))
gen.apply_action()
print(f"Generator power: {gen.state.features['GenOutput'].p_mw:.1f} MW")
print(f"Generator cost: ${gen.state.features['CostSafety'].cost:.0f}")

Generator power: 8.0 MW
Generator cost: $400


## Step 3: Building a Coordinator Agent (Microgrid)

Coordinators manage field agents. The key pattern is:

**Pass pre-initialized subordinates to the constructor:**

```python
# Create device agents first
devices = {
    'gen_1': SimpleGenerator(agent_id='gen_1', ...),
    'bat_1': SimpleBattery(agent_id='bat_1', ...),
}

# Then create coordinator with subordinates
coordinator = CoordinatorAgent(
    agent_id='mg_1',
    subordinates=devices,  # Pass pre-initialized agents
    ...
)
```

In [5]:
from heron.agents.coordinator_agent import CoordinatorAgent
from heron.protocols.vertical import VerticalProtocol


@dataclass(slots=True)
class MicrogridMetrics(FeatureProvider):
    """Aggregated microgrid metrics."""
    visibility: ClassVar[Sequence[str]] = ['owner', 'system']
    total_generation_MW: float = 0.0
    total_load_MW: float = 0.0
    net_power_MW: float = 0.0


class SimpleMicrogrid(CoordinatorAgent):
    """Microgrid coordinator - manages battery + generator.
    
    Key Pattern: Subordinates are passed to constructor, not created internally.
    """
    
    def __init__(
        self,
        agent_id: str,
        subordinates: dict,  # Pre-initialized device agents
        load_MW: float = 3.0,
        upstream_id: Optional[str] = None,
    ):
        self._load_MW = load_MW
        
        # Coordinator-level features
        features = [
            MicrogridMetrics(total_load_MW=load_MW),
        ]
        
        super().__init__(
            agent_id=agent_id,
            features=features,
            subordinates=subordinates,  # Pass to parent
            upstream_id=upstream_id,
            protocol=VerticalProtocol(),
            tick_config=TickConfig.deterministic(tick_interval=60.0),
        )
    
    def compute_local_reward(self, local_state: dict) -> float:
        """Coordinator reward from aggregated subordinate state.
        
        Args:
            local_state: Dict mapping feature_name -> np.array vector
                         (populated via visibility rules, not direct access)
        
        Returns:
            Reward value (higher is better)
        """
        total_cost = 0.0
        total_safety = 0.0
        
        # Use local_state populated via visibility rules
        if 'CostSafety' in local_state:
            metrics = local_state['CostSafety']
            total_cost = float(metrics[0])
            total_safety = float(metrics[1])
        
        return -total_cost - total_safety

In [6]:
# Create device agents FIRST
mg_devices = {
    'mg_1_bat': SimpleBattery(
        agent_id='mg_1_bat',
        upstream_id='mg_1',
    ),
    'mg_1_gen': SimpleGenerator(
        agent_id='mg_1_gen',
        upstream_id='mg_1',
    ),
}

# THEN create coordinator with subordinates
mg = SimpleMicrogrid(
    agent_id='mg_1',
    subordinates=mg_devices,
    load_MW=3.0,
)

print(f"Microgrid: {mg.agent_id}")
print(f"Subordinates: {list(mg.subordinates.keys())}")
print(f"Coordinator features: {list(mg.state.features.keys())}")

Microgrid: mg_1
Subordinates: ['mg_1_bat', 'mg_1_gen']
Coordinator features: ['MicrogridMetrics']


## Step 4: Building a System Agent (Grid Operator)

SystemAgent is the **top level (L3)**. It manages coordinators.

**Same pattern:** Pass pre-initialized coordinators to constructor.

```python
# Create coordinators first
microgrids = {
    'mg_1': SimpleMicrogrid(...),
    'mg_2': SimpleMicrogrid(...),
}

# Then create system agent
system = SystemAgent(
    agent_id='grid_system',
    subordinates=microgrids,
)
```

In [7]:
from heron.agents.system_agent import SystemAgent


@dataclass(slots=True)
class SystemFrequency(FeatureProvider):
    """Grid frequency - system-level metric."""
    visibility: ClassVar[Sequence[str]] = ['system']
    frequency_hz: float = 60.0
    nominal_hz: float = 60.0

    def vector(self) -> np.ndarray:
        deviation = (self.frequency_hz - self.nominal_hz) / self.nominal_hz
        return np.array([deviation], dtype=np.float32)


class SimpleGridSystem(SystemAgent):
    """System agent - manages multiple microgrids.
    
    Key Pattern: Subordinates (coordinators) passed to constructor.
    """
    
    def __init__(
        self,
        agent_id: str = 'grid_system',
        subordinates: Optional[dict] = None,
    ):
        features = [
            SystemFrequency(frequency_hz=60.0),
        ]
        
        super().__init__(
            agent_id=agent_id,
            features=features,
            subordinates=subordinates or {},
            tick_config=TickConfig.deterministic(tick_interval=300.0),
        )
    
    def compute_local_reward(self, local_state: dict) -> float:
        """System reward from aggregated coordinator state.
        
        Args:
            local_state: Dict mapping feature_name -> np.array vector
                         (populated via visibility rules, not direct access)
        
        Returns:
            Reward value (higher is better)
        """
        total_cost = 0.0
        total_safety = 0.0
        
        # Use local_state populated via visibility rules
        if 'MicrogridMetrics' in local_state:
            metrics = local_state['MicrogridMetrics']
            # Extract relevant metrics from the vector
            pass
        
        if 'CostSafety' in local_state:
            metrics = local_state['CostSafety']
            total_cost = float(metrics[0])
            total_safety = float(metrics[1])
        
        return -total_cost - total_safety

In [8]:
# Build the complete hierarchy bottom-up

# 1. Create devices for microgrid 0
mg0_devices = {
    'mg_0_bat': SimpleBattery(agent_id='mg_0_bat', upstream_id='mg_0'),
    'mg_0_gen': SimpleGenerator(agent_id='mg_0_gen', upstream_id='mg_0'),
}

# 2. Create devices for microgrid 1
mg1_devices = {
    'mg_1_bat': SimpleBattery(agent_id='mg_1_bat', upstream_id='mg_1'),
    'mg_1_gen': SimpleGenerator(agent_id='mg_1_gen', upstream_id='mg_1'),
}

# 3. Create coordinators
microgrids = {
    'mg_0': SimpleMicrogrid(agent_id='mg_0', subordinates=mg0_devices, upstream_id='grid_system'),
    'mg_1': SimpleMicrogrid(agent_id='mg_1', subordinates=mg1_devices, upstream_id='grid_system'),
}

# 4. Create system agent
system = SimpleGridSystem(
    agent_id='grid_system',
    subordinates=microgrids,
)

print(f"System: {system.agent_id}")
print(f"Coordinators: {list(system.subordinates.keys())}")
for mg_id, mg in system.subordinates.items():
    print(f"  {mg_id} -> Devices: {list(mg.subordinates.keys())}")

System: grid_system
Coordinators: ['mg_0', 'mg_1']
  mg_0 -> Devices: ['mg_0_bat', 'mg_0_gen']
  mg_1 -> Devices: ['mg_1_bat', 'mg_1_gen']


## Using the PowerGrid Case Study Agents

The `powergrid` module provides production-ready implementations:

```python
from powergrid import Generator, ESS, PowerGridAgent, GridSystemAgent

# Create device agents
devices = {
    'gen_1': Generator(
        agent_id='gen_1',
        bus='bus_1',
        p_min_MW=1.0,
        p_max_MW=10.0,
        cost_curve_coefs=(0.01, 1.0, 0.0),
    ),
    'ess_1': ESS(
        agent_id='ess_1',
        bus='bus_2',
        capacity_MWh=10.0,
        p_min_MW=-5.0,
        p_max_MW=5.0,
    ),
}

# Create coordinator
grid = PowerGridAgent(
    agent_id='grid_1',
    subordinates=devices,
)

# Create system agent
system = GridSystemAgent(
    agent_id='system',
    subordinates={'grid_1': grid},
)
```

## Summary: The HERON Patterns

### FieldAgent Pattern
```python
class MyDevice(FieldAgent):
    def __init__(self, agent_id, ...):
        features = [MyFeature(...)]  # Create features
        super().__init__(agent_id=agent_id, features=features, ...)
    
    def init_action(self, features) -> Action:
        action = Action()
        action.set_specs(dim_c=1, range=(...))  # Define space
        return action
    
    def set_action(self, action):              # Store action
        self.action.set_values(c=action)
    
    def set_state(self, **kwargs):             # Update features
        # Denormalize action, apply physics, update features
        pass
    
    def apply_action(self):                    # Apply action
        self.set_state()
    
    def compute_local_reward(self, local_state) -> float:
        return -cost - safety
```

### CoordinatorAgent Pattern
```python
# Create devices first, then pass to coordinator
devices = {'dev_1': Device1(...), 'dev_2': Device2(...)}

coordinator = CoordinatorAgent(
    agent_id='coord_1',
    subordinates=devices,  # Pre-initialized agents
    features=[CoordFeature(...)],
    protocol=VerticalProtocol(),
)
```

### SystemAgent Pattern
```python
# Create coordinators first, then pass to system
coordinators = {'mg_1': Microgrid1(...), 'mg_2': Microgrid2(...)}

system = SystemAgent(
    agent_id='system',
    subordinates=coordinators,  # Pre-initialized coordinators
    features=[SystemFeature(...)],
)
```

## Key Takeaways

| Agent Type | Level | Constructor Pattern | Required Methods |
|------------|-------|---------------------|------------------|
| `FieldAgent` | L1 | `features=[...]` | `init_action`, `set_action`, `set_state`, `apply_action`, `compute_local_reward` |
| `CoordinatorAgent` | L2 | `subordinates={...}` | (optional overrides) |
| `SystemAgent` | L3 | `subordinates={...}` | (optional overrides) |

**The HERON contract:**
- **Build hierarchy bottom-up** — Create devices, then coordinators, then system
- **Pass subordinates to constructor** — No `_build_subordinates()` method
- **Features as dict** — Stored as `{feature_name: feature}`, accessed via `state.features['Name']`
- **Agents don't have `step()`** — Physics lives in the environment

**Two execution modes:**
- **Synchronous (training):** `set_action()` → `apply_action()`
- **Event-driven (testing):** Scheduler calls `tick()` → handlers manage flow

---

**Next:** [03_building_environment.ipynb](03_building_environment.ipynb) — Create the environment (where physics lives)